In [2]:
# Imports
import json
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from pygeocoder import Geocoder
from sklearn.preprocessing import StandardScaler
from geopy.geocoders import GeocodeFarm, Nominatim
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from datetime import datetime

In [4]:
clean_df = pd.read_csv("../data/cleaned_government_data.csv")
clean_df.head(5)

,Major Class,Month of Travel Date,From,To,Sum of Net Tickets,Sum of Total $
0,Business Class,Jan,Calgary,Ottawa,2,6045.62
1,Business Class,Jan,Calgary,Victoria,1,740.60
2,Business Class,Jan,Campbell River,Ottawa,1,3482.85
3,Business Class,Jan,Charlottetown,Calgary,1,2807.24
4,Business Class,Jan,Charlottetown,Ottawa,1,737.35


## Origin, Destination encoding

In [8]:
geo = Nominatim()
geo_farm = GeocodeFarm()

cities=list(clean_df["From"].unique())
cities.extend(list(clean_df["To"].unique()))
cities = list(set(cities))

/Volumes/Macintosh HDD/Tech_Hackathon/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


In [10]:
len(cities)

264

In [11]:
location_details = {}

In [12]:
for each_city in cities:
    try:
        location = geo.geocode(each_city + ", Canada")
    except:
        try:
            location = geo_farm.geocode(each_city + ", Canada")
        except:
            continue
    try:
        location_details[each_city] = {
            "latitude": location[1][0],
            "longitude": location[1][1]
        }
    except:
        continue

In [19]:
location_details["St Theris Point"] = {
    "latitude": 53.8328,
    "longitude": -94.8497
}

In [20]:
location_details["Ganges Harbor"] = {
    "latitude": 48.8556,
    "longitude": -123.4965
}

In [26]:
location_details["Pender Harbor"] = {
    "latitude": 49.6271,
    "longitude": -124.0359
}

In [27]:
location_details["Points North Landing"] = {
    "latitude": 58.2701,
    "longitude": -104.0814
}

In [24]:
location_details["PITTS MEADOW  BC"]

{'latitude': 42.2579, 'longitude': -83.05942}

In [28]:
with open("../data/city_coordinates.json", "w") as f:
    json.dump(location_details, f)

In [40]:
def load_coordinates():
    # Loading coords from JSON
    with open("../data/city_coordinates.json") as f:
        coords = json.load(f)
    return coords

In [41]:
def transform_coordinates(clean_df):
    # Loading coordinates
    location_details = load_coordinates()
    # Origin lat, lon and destination lat, long figured
    clean_df[["From_lat","From_lon"]] = clean_df["From"].apply(lambda x: pd.Series([location_details[x]["latitude"],location_details[x]["longitude"]]))
    clean_df[["To_lat","To_lon"]] = clean_df["To"].apply(lambda x: pd.Series([location_details[x]["latitude"],location_details[x]["longitude"]]))
    clean_df.drop(["From", "To"], axis=1, inplace=True)
    return clean_df

In [42]:
def convert_to_radians(clean_df, cols=None):
    # Converting degree to rads
    clean_df[cols] = np.radians(clean_df[cols])
    return clean_df

In [44]:
clean_df = transform_coordinates(clean_df)
clean_df = convert_to_radians(clean_df, cols=["From_lat", "From_lon", "To_lat", "To_lon"])

In [45]:
clean_df.head(5)

,Major Class,Month of Travel Date,Sum of Net Tickets,Sum of Total $,From_lat,From_lon,To_lat,To_lon
0,Business Class,Jan,2,6045.62,0.891050,-1.990768,0.792748,-1.321045
1,Business Class,Jan,1,740.60,0.891050,-1.990768,0.845234,-2.153125
2,Business Class,Jan,1,3482.85,0.873067,-2.185923,0.792748,-1.321045
3,Business Class,Jan,1,2807.24,0.806952,-1.101878,0.891050,-1.990768
4,Business Class,Jan,1,737.35,0.806952,-1.101878,0.792748,-1.321045


## Feature Engineering: Distance between origin and destination

In [46]:
def haversine(clean_df):
    # Distance metric to calculate between two coordinate points
    from_lat, from_long, to_lat, to_long = clean_df["From_lat"], clean_df["From_lon"], clean_df["To_lat"], clean_df["To_lon"]
    radius_of_earth = 6378.1  # In km
    lat_delta = to_lat - from_lat
    lon_delta = to_long - from_long
    d = np.sin(lat_delta*0.5)**2 + np.cos(from_lat) * np.cos(to_lat) * np.sin(lon_delta*0.5)**2
    haversine_values = 2 * radius_of_earth * np.arcsin(np.sqrt(d))
    return haversine_values

In [47]:
def calculate_distance(clean_df):
    # Add a distance column to denote distance between origin and destination
    clean_df["distance"] = clean_df.apply(haversine, axis=1)
    return clean_df

In [48]:
clean_df = calculate_distance(clean_df)
clean_df.head(5)

,Major Class,Month of Travel Date,Sum of Net Tickets,Sum of Total $,From_lat,From_lon,To_lat,To_lon,distance
0,Business Class,Jan,2,6045.62,0.891050,-1.990768,0.792748,-1.321045,2878.608298
1,Business Class,Jan,1,740.60,0.891050,-1.990768,0.845234,-2.153125,729.555861
2,Business Class,Jan,1,3482.85,0.873067,-2.185923,0.792748,-1.321045,3677.013367
3,Business Class,Jan,1,2807.24,0.806952,-1.101878,0.891050,-1.990768,3708.065355
4,Business Class,Jan,1,737.35,0.806952,-1.101878,0.792748,-1.321045,977.226751


## Major Class

In [49]:
clean_df.groupby(["Major Class"])[["Sum of Net Tickets", "Sum of Total $"]].sum()['Sum of Total $']/clean_df.groupby(["Major Class"])[["Sum of Net Tickets", "Sum of Total $"]].sum()['Sum of Net Tickets']

Major Class
Business Class     1845.703896
Economy             486.790989
First Class         235.342727
Premium Economy    1511.283000
dtype: float64

### Inference:
- Although First Class gives a very low mean price, it is logically the most expensive way to travel via air.
- Upon closer examination, it was seen that the first class records observed in the data was for a very short distance.
- Thus, as per logic, it is being label encoded with First class ranking the highest and Economy ranking the lowest.
- This might give a slightly lower accuracy for our dataset but is more interpretable.

In [50]:
def dump_fe_pkl(model, col_name):
    with open(f"../fe_models/fe_{col_name}.pkl", "wb") as f:
        pickle.dump(model, f)

In [51]:
def load_fe_pkl(col_name):
    with open(f"../fe_models/fe_{col_name}.pkl", "rb") as f:
        model=pickle.load(f)
    return model

In [52]:
def label_encode(clean_df, col_name=None, use_pre_trained=False):
    if not use_pre_trained:
        model = LabelEncoder()
        model.fit(clean_df[col_name])
        dump_fe_pkl(model, col_name)
    model=load_fe_pkl(col_name)
    clean_df[col_name] = model.transform(clean_df[col_name])
    return clean_df

In [53]:
def custom_label_encode(clean_df, mappings):
    clean_df = clean_df.replace(mappings)
    return clean_df

In [54]:
mappings = {
    "Major Class":{
        "Economy": 1,
        "Premium Economy": 2,
        "Business Class": 3,
        "First Class": 4
    }
}
clean_df = custom_label_encode(clean_df, mappings)

In [55]:
clean_df.head(5)

,Major Class,Month of Travel Date,Sum of Net Tickets,Sum of Total $,From_lat,From_lon,To_lat,To_lon,distance
0,3,Jan,2,6045.62,0.891050,-1.990768,0.792748,-1.321045,2878.608298
1,3,Jan,1,740.60,0.891050,-1.990768,0.845234,-2.153125,729.555861
2,3,Jan,1,3482.85,0.873067,-2.185923,0.792748,-1.321045,3677.013367
3,3,Jan,1,2807.24,0.806952,-1.101878,0.891050,-1.990768,3708.065355
4,3,Jan,1,737.35,0.806952,-1.101878,0.792748,-1.321045,977.226751


In [56]:
def reorder_cols(clean_df, col_order=[]):
    return clean_df[col_order]

In [57]:
clean_df = reorder_cols(clean_df, col_order=['Major Class', 'Month of Travel Date', 'Sum of Net Tickets', 'From_lat', 'From_lon', 'To_lat', 'To_lon',
       'distance', 'Sum of Total $'])
clean_df.head(5)

,Major Class,Month of Travel Date,Sum of Net Tickets,From_lat,From_lon,To_lat,To_lon,distance,Sum of Total $
0,3,Jan,2,0.891050,-1.990768,0.792748,-1.321045,2878.608298,6045.62
1,3,Jan,1,0.891050,-1.990768,0.845234,-2.153125,729.555861,740.60
2,3,Jan,1,0.873067,-2.185923,0.792748,-1.321045,3677.013367,3482.85
3,3,Jan,1,0.806952,-1.101878,0.891050,-1.990768,3708.065355,2807.24
4,3,Jan,1,0.806952,-1.101878,0.792748,-1.321045,977.226751,737.35


## Month of Travel Date

In [ ]:
clean_df.groupby(["Month of Travel Date"])[["Sum of Net Tickets", "Sum of Total $"]].describe()

### Inference:
- One-hot encode months because there is no significant rise in price between the months.
- This was determined by diving the mean of Sum of Total with the Sum of Net Tickets for every month.

In [58]:
def compute_avg_price(clean_df):
    clean_df["price"] = clean_df["Sum of Total $"]/clean_df["Sum of Net Tickets"]
    clean_df.drop(["Sum of Total $", "Sum of Net Tickets"], axis=1, inplace=True)
    return clean_df

In [59]:
clean_df = compute_avg_price(clean_df)

In [61]:
def ohe_encode(clean_df, categories=None, use_pre_trained=False):
    if not use_pre_trained:
        model = make_column_transformer((categories, OneHotEncoder()), remainder="passthrough")
        clean_df_data, clean_df_labels = model.fit_transform(clean_df.iloc[:, :-1]), np.array(clean_df.iloc[:, -1])
        dump_fe_pkl(model, "ohe_model")
        return clean_df_data, clean_df_labels
    model=load_fe_pkl("ohe_model")
    clean_df = model.transform(clean_df)
    return clean_df

### Also, splitting the encoded data into features and targets

In [62]:
X, y = ohe_encode(clean_df, categories=["Month of Travel Date"])

/Volumes/Macintosh HDD/Tech_Hackathon/venv/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:778: DeprecationWarning: `make_column_transformer` now expects (transformer, columns) as input tuples instead of (columns, transformer). This has been introduced in v0.20.1. `make_column_transformer` will stop accepting the deprecated (columns, transformer) order in v0.22.
  warnings.warn(message, DeprecationWarning)


## Cross Validation for model selection

In [65]:
# Use cross validation to find a model that gives the least error (using mean absolute error)

# Linear Regression on the model
lin_reg_cv_score = cross_val_score(LinearRegression(), X, y, scoring="neg_mean_absolute_error", cv=10, n_jobs=-1)
print(f"Mean absolute error with Linear Regression is: {lin_reg_cv_score}\n")

# Random Forest Regressor
forest_reg_cv_score = cross_val_score(RandomForestRegressor(), X, y, scoring="neg_mean_absolute_error", cv=10, n_jobs=-1)
print(f"Mean absolute error with Random Forest Regressor is: {forest_reg_cv_score}\n")

# XGBoost Regressor
xgb_reg_cv_score = cross_val_score(XGBRegressor(), X, y, scoring="neg_mean_absolute_error", cv=10, n_jobs=-1)
print(f"Mean absolute error with XGBoost Regressor is: {xgb_reg_cv_score}")

Mean absolute error with Linear Regression is: [-525.44973477 -380.38618915 -333.4080112  -329.64843716 -331.48375537
 -304.30249959 -333.78092374 -351.60640486 -330.50286717 -301.9693914 ]

Mean absolute error with Random Forest Regressor is: [-491.85513025 -262.07968083 -260.71133562 -241.24452713 -259.60450712
 -242.69164273 -250.25857782 -248.5783139  -248.31922422 -232.48985057]

Mean absolute error with XGBoost Regressor is: [-545.88526887 -306.41334643 -290.2479453  -286.53016615 -282.98878098
 -283.28770994 -286.74360486 -284.80685806 -287.16324272 -277.83623674]


## GRIDSEARCH for Hyperparameter Tuning

In [ ]:
# Since Random Forest Regressor gives the lowest error among all models, we will use
# GridSearchCV to tune the hyper-parameters for RF Regressor and minimize the error

print(datetime.now())

rf_parameters = {'n_estimators':[120, 150, 200],
                 'max_depth':[20, 30, 50], 'min_samples_leaf':[1, 3, 5]}
rf_gsc = GridSearchCV(RandomForestRegressor(), param_grid=rf_parameters, scoring="neg_mean_squared_error", cv=5, n_jobs=-1, verbose=True)
grid_search_result = rf_gsc.fit(X, y)
print(f"The best set of hyper-parameters are: {grid_search_result.best_params_}")

print(datetime.now())

In [ ]:
with open("../trained_models/model.pkl", "wb") as f:
    pickle.dump(grid_search_result.best_estimator_, f)

## Function to preprocess data from front-end

In [66]:
def preprocessing(clean_df):
    # Transforming "From" and "To" destinations to coordinates
    clean_df = transform_coordinates(clean_df)
    # Transforming coordinates to radians
    clean_df = convert_to_radians(clean_df, cols=["From_lat", "From_lon", "To_lat", "To_lon"])
    # Computing haversine between origin and destination
    clean_df = calculate_distance(clean_df)
    # Transforming "Major Class" using Label Encoder
    clean_df = custom_label_encode(clean_df, mappings)
    # Transforming "Month of Travel Date" using one-hot encoder
    clean_arr = ohe_encode(clean_df, categories=["Month of Travel Date"], use_pre_trained=True)
    return clean_arr

In [67]:
sample = pd.DataFrame([["Vancouver","Halifax","Economy","Dec"]], columns=["From", "To", "Major Class", "Month of Travel Date"])
sample=preprocessing(sample)

In [68]:
sample

array([[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1,
        0.8597644157848011, -2.1487438332502213, 0.7792653905652056,
        -1.1097841628192082, 4434.102538202029]], dtype=object)